In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
import numpy as np
import json
from llama_index import Document
import openai

openai.api_key = os.environ.get("OPENAI_API_KEY")
from llama_index.vector_stores import PineconeVectorStore,MilvusVectorStore
from PyPDF2 import PdfReader
from llama_index import GPTVectorStoreIndex, StorageContext, ServiceContext
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import VectorStoreIndex, StorageContext

In [4]:
import time
from PyPDF2 import PdfReader
from PIL import Image

import os


# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
import openai
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import ChatGLM
from langchain.prompts import PromptTemplate

In [5]:
uri=os.environ.get("uri")
token=os.environ.get("token")
collection_name=os.environ.get("MILVUS_COLLECTION_NAME")
dim=os.environ.get("VECTOR_DIMENSION")
vector_field=os.environ.get("VECTOR_FIELD")
doc_id_field=os.environ.get("DOC_ID_FIELD")
text_key=os.environ.get("TEXT_KEY")
pincone_env=os.environ.get("PINCONE_ENV")
pincone_key=os.environ.get("PINCONE_KEY")

In [6]:
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [7]:
# # default endpoint_url for a local deployed ChatGLM api server
# endpoint_url = "https://5448-35-227-160-165.ngrok-free.app"

# # direct access endpoint in a proxied environment
# # os.environ['NO_PROXY'] = '127.0.0.1'

# llm = ChatGLM(
#     endpoint_url=endpoint_url,
#     max_token=80000,
#     history=[],
#     top_p=0.9,
# )

In [8]:

import pinecone



# initialize connection to pinecone
pinecone.init(
    api_key=os.environ.get('PINCONE_KEY'),
    environment=os.environ.get('PINCONE_ENV')
)

# create the index if it does not exist already
index_name = 'llama-index-intro'
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=768,
        metric='cosine'
    )

# connect to the index
pinecone_index = pinecone.Index(index_name)

/home/nikhil/Documents/Github/Sumquery-PDF-AiSummarizer/venv/lib/python3.8/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [8]:
vector_store_pinecone = PineconeVectorStore(pinecone_index=pinecone_index)

In [9]:

# setup our storage (vector db)
storage_context = StorageContext.from_defaults(
    vector_store=pinecone_index
)
# setup the index/query process, ie the embedding model (and completion if used)
embed_model = HuggingFaceEmbeddings()
service_context = ServiceContext.from_defaults(embed_model=embed_model)


In [10]:
index=VectorStoreIndex.from_vector_store(vector_store=vector_store_pinecone,service_context=service_context)

In [41]:
query_engine = index.as_query_engine()
res = query_engine.query("List the top 15 main scenes in sequence covering the entire script with brief descriptions in 50 words.")

In [42]:
print(res.response)

1. Trudy and Max overpower a guard and free a prisoner from a cell.
2. Another trooper appears, but Trudy takes him down while Max knocks out the first trooper.
3. Trudy kisses Norm as he escapes from the cell.
4. Jake grabs a sidearm from a fallen trooper and thanks Max and Trudy.
5. Jake asks the group if it's time for a revolution.
6. Grace expresses her freedom and Trudy taps Jake's fist.
7. The group reaches a utility corridor and Jake pumps a chair while the others put on exopacks.
8. Norm points out a herd of sturmbest, massive six-legged creatures.
9. Hundreds of purple winged creatures take flight from a lake.
10. The Samson flies over a waterfall, with Trudy performing daring maneuvers.
11. Wainfleet encourages Chacon to "get some" during the flight.
12. The Samson lands in a clearing, causing the fern-like grass to be flattened by the rotor-wash.
13. Jake removes a massive door gun from its mount.
14. The group prepares for the next scene or action.
15. The script continues 

In [1]:
from langchain.chains import RetrievalQA

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)